In [19]:
import numpy as np
import arviz as az
import pandas as pd

# import pymc3 as pm
import pickle
import matplotlib.pyplot as plt
from google.protobuf import text_format
from scipy.stats import multivariate_normal as mvn
from scipy.stats import skewnorm
from scipy.stats import norm
from scipy.interpolate import griddata
import sys
sys.path.append('/home/lorenzo/Documents/Tesi/github_repos/pp_mix/')
sys.path.append('/home/lorenzo/Documents/Tesi/github_repos/pp_mix/pp_mix')

import pp_mix.protos.py.params_pb2 as params_pb2
from pp_mix.interface import ConditionalMCMC, cluster_estimate
from pp_mix.utils import loadChains, to_numpy, to_proto
from pp_mix.protos.py.state_pb2 import MultivariateMixtureState, EigenVector, EigenMatrix
from pp_mix.protos.py.params_pb2 import Params

In [18]:
params_file = "data/Eyes_data/resources/sampler_params.asciipb"


In [20]:
df = pd.read_excel("41598_2021_2025_MOESM2_ESM.xlsx")


In [30]:
# extract only right eyes (OD)
df_od = df.loc[df['Masked_Id_Age1'].str.contains("OD", case=True)].set_index('Masked_Id_Age1')
# clustering obtained by cifu (method used in the paper)
cifu_clustering = df_od['Cluster_Id_Age1']

# THIS IS THE DATA USED IN THE ALGORITHM, THEN CONVERTED TO NUMPY ARRAY
df_od = df_od[df_od.columns[1:]]
df_od.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180
Masked_Id_Age1,,,,,,,,,,,,,,,,,,,,,
13435OD,0.003646,0.003648,0.003655,0.003665,0.003679,0.003696,0.003715,0.003737,0.003762,0.003791,...,0.003696,0.003691,0.003686,0.003680,0.003674,0.003668,0.003662,0.003656,0.003651,0.003647
13564OD,0.002993,0.003013,0.003042,0.003078,0.003116,0.003152,0.003181,0.003205,0.003225,0.003243,...,0.002988,0.002996,0.003002,0.003004,0.003002,0.002996,0.002989,0.002982,0.002979,0.002982
13587OD,0.002907,0.002940,0.002982,0.003034,0.003097,0.003177,0.003275,0.003396,0.003538,0.003699,...,0.002913,0.002900,0.002885,0.002869,0.002855,0.002845,0.002844,0.002850,0.002862,0.002881
13588OD,0.003334,0.003378,0.003451,0.003552,0.003674,0.003811,0.003956,0.004098,0.004232,0.004352,...,0.003925,0.003836,0.003745,0.003655,0.003567,0.003488,0.003420,0.003367,0.003332,0.003320
13743OD,0.004369,0.004352,0.004336,0.004315,0.004286,0.004247,0.004202,0.004156,0.004112,0.004072,...,0.004831,0.004795,0.004747,0.004690,0.004628,0.004567,0.004511,0.004462,0.004422,0.004392


In [32]:
# NUMPY ARRAY OF DATA
data = df_od.to_numpy()